## Tarea semana 4: Clustering de empleados

### Caso de Negocio: Creative HR

Creative HR es una empresa de generación de contenidos en redes sociales con oficinas en varios países. La empresa se enorgullece de su cultura dinámica y creativa, y se esfuerza por atraer y retener a los mejores talentos. Sin embargo, en los últimos años, Creative HR ha experimentado un aumento en la tasa de rotación de empleados. El director de recursos humanos está preocupado por el impacto negativo que la alta tasa de rotación de empleados está teniendo en la empresa. El costo de reemplazar a los empleados que se van es alto, y la pérdida de conocimiento y experiencia también puede ser significativa. El director quiere identificar las causas de la deserción de empleados y desarrollar estrategias para reducirla.  Considera que eres parte del equipo de analistas de Creative HR y se te ha encomendado encontrar posibles causas y soluciones a este problema.

**Objetivo del Análisis de Clustering para Recursos Humanos**

1. Segmentación de Empleados

- Identificar grupos homogéneos de empleados con características similares.
- Agrupar empleados según habilidades, experiencia, desempeño y niveles de satisfacción.
- Personalizar estrategias de desarrollo y retención para cada segmento.

2. Análisis de Perfiles de Talento

- Clasificar candidatos o empleados según habilidades técnicas, habilidades blandas y competencias específicas.
- Asignar roles y responsabilidades de acuerdo con los perfiles de talento identificados.
- Mejorar la alineación entre las habilidades del empleado y los requisitos del puesto.

3. Gestión de Carreras y Desarrollo

- Identificar grupos de empleados con trayectorias profesionales similares.
- Desarrollar planes de carrera personalizados para diferentes segmentos.
- Facilitar el crecimiento profesional anticipando las necesidades de capacitación y desarrollo.

4. Clima Laboral y Compromiso

- Agrupar empleados según niveles de satisfacción, compromiso y factores de clima laboral.
- Identificar áreas de mejora en función de los patrones de agrupación.
- Personalizar estrategias para aumentar el compromiso y la satisfacción.

5. Análisis de Rotación de Personal

- Identificar grupos de empleados con mayor riesgo de rotación.
- Desarrollar estrategias de retención específicas para cada segmento.
- Anticipar y abordar problemas de retención de manera proactiva.

Entre otros.


- La empresa Creative HR está analizando los factores de pérdida de personal (**Attrition**). 
- El departamento de recursos humanos ha recopilado información demográfica y de educación de sus actuales y antiguos empleados.
- **El departamento de recursos humanos pretende agrupar grupos de empleados de acuerdo a sus características en común y analizar los grupos resultantes.**

### OPCIÓN 1: LOW-CODE Y RÉPLICA DE NOTEBOOK

**Archivo de datos:**  

- [CS_data.csv](https://raw.githubusercontent.com/marsgr6/EN-online/8a1cee296279c274d8f05f3905fce97c1f7636a2/data/CHR_data.csv)

- Lea el archivo csv. 
  
```Python
import pandas as pd
import seaborn as sns

data = pd.read_csv("../data/CHR_data.csv")  # si la estructura es la del repositorio de GitHub

# si está trabajando en local y el archivo de datos y el notebook están en el mismo directorio
# data = pd.read_csv("CHR_data.csv") 

# También puede leer el archivo desde el repositorio de GitHub.
#data = pd.read_csv("https://raw.githubusercontent.com/marsgr6/EN-online/8a1cee296279c274d8f05f3905fce97c1f7636a2/data/CHR_data.csv")
```

**Indicaciones:**

1. **(2pts)** Preprocesamiento de datos
- Filtre los datos, conserve solamente las columnas numéricas.
```Python
data_cl = data.select_dtypes(include='number')
```
- Remueva columnas con varianza cero.

```Python
data_cl = data_cl.loc[:, data_cl.var() > 0.0]
```

- Estandarice los datos usando escalado estándar.

```Python
# standardizing the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_cl)
```

2.  **(2pts)** Utilice K-means y realice un análisis de codo para decidir cuántos clusters usar.

```Python
from sklearn.cluster import KMeans

# fitting multiple k-means algorithms and storing the values in an empty list
SSE = []
test_clusters = 10
for cluster in range(1,test_clusters+1):
    kmeans = KMeans(n_clusters=cluster, n_init="auto")
    kmeans.fit(data_scaled)
    SSE.append(kmeans.inertia_)

# converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,test_clusters+1), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
```

<img src="https://raw.githubusercontent.com/marsgr6/r-scripts/master/imgs/elbow_attrition.png"  width="600"/>

3.  **(2pts)** Use PCA para visualizar los clusters generados.
  - Visualice usando las priemeras 2 componentes principales.

```Python
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

principalComponents = pca.fit_transform(data_scaled)

pc_transformation = pd.DataFrame(data = principalComponents,
                           columns = ['PC1', 'PC2'])

kmeans = KMeans(n_clusters=2, n_init="auto").fit(data_scaled)

pc_transformation['Cluster'] = pd.Categorical(kmeans.labels_)

sns.scatterplot(data=pc_transformation, x="PC1", y="PC2", hue="Cluster")
```

4. **(2pts)** Realice un análisis descriptivo y decida entre  $k\in \{2, 3, 4, 5\}$.
  - A continuación tiene un ejemplo para visualizar 5 clusters usando PCA.
  
  - **Replique el código del punto 3 cambiando el número de clústeres.** 

<img src="https://raw.githubusercontent.com/marsgr6/r-scripts/master/imgs/kmeans5_attrition.png" width="600"/>

  - Análisis descriptivo (gráficos de violín):
  
```Python
sns.violinplot(data=data, x=kmeans.labels_, y="TotalWorkingYears", 
               hue="Attrition", split=True, inner="quartil")
```

<img src="https://raw.githubusercontent.com/marsgr6/r-scripts/master/imgs/cl5_violin_attrition.png" width="600"/>

5. **(2pts)** De conclusiones.

**ANEXO**

- Random Forest (RF) le puede ayudar a identificar qué variables son importantes para generar los grupos, estas son las variables que quiere utilizar en su análisis descriptivo para entender los grupos.
- En este caso se muestra un ejemplo de features importantes según RF para 5 clusters.

<img src="https://raw.githubusercontent.com/marsgr6/r-scripts/master/imgs/rf5_attrition.png" width="600"/>

### OPCIÓN 2: REPORTE EJECUTIVO PARA GERENCIA

- **Elabore un informe ejecutivo** discutiendo las figuras y resultados anteriores, puntos 1 a 5 de la Opción Low Code.

El reporte ejecutivo debe tener una extensión máxima de 5 páginas. Crea un documento en word que incluya lo siguiente:

  1. Descripción general del problema empresarial: explica el contexto o problema empresarial y el enfoque analítico. 
  2. Diccionario de variables o base de datos: describe los datos utilizados para el análisis. 
  3. Análisis exploratorio de datos: utiliza los gráficos del notebook para interpretar los datos utilizados acorde al contexto empresarial.
  4. Interpretación de metodologías: explica porque los modelos de aprendizaje no supervisado implementados ayudan a resolver el problema empresarial. También incluye una interpretación de los resultados de dichos modelos con detalle. Parte de los resultados del análisis de clustering están disponibles en las secciones 1 a 5 de la Opción Low Code. Para obtener los resultados adicionales ejecute el script en la celda de código siguiente. 
  5. Conclusión y Recomendaciones: ofrece recomendaciones gerenciales y sugerencias estratégicas que permita a la empresa tomar decisiones.

La explicación en todas las secciones del reporte ejecutivo debe ser detallado, justificado y relacionado al contexto o problema empresarial. Está información será utilizada por el director de recursos humanos de Creative HR y un reporte ambiguo podría causar toma de decisiones incorrectas y pérdida de dinero.

- A continuación puede ejecutar el script para construir los gráficos para diferntes números de clusters $k\in \{2, 3, 4, 5\}$. 
- Descargue el archivo del script: [viz.py](https://raw.githubusercontent.com/marsgr6/EN-online/main/viz.py).
  - Construya su informe a partir de estos gráficos descriptivos.
  - Vea dos ejemplos de ejecución:
  
<img src="https://raw.githubusercontent.com/marsgr6/r-scripts/master/imgs/clus_viz_ts4.png" width="600"/>

<img src="https://raw.githubusercontent.com/marsgr6/r-scripts/master/imgs/pc1pc2viz.png" width="600"/>

In [2]:
## import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from viz import *

# Leear archivo de datos
data_clusters = pd.read_csv("https://raw.githubusercontent.com/marsgr6/r-scripts/master/data/hr_data_clusters.csv")

# Transformar las (últimas 4) columnas con los clusters en categóricas 
data_clusters.iloc[:,-4:] = data_clusters.iloc[:,-4:].astype(str)

all_data = {"HR Clusters": data_clusters
           }

plot_data(all_data)

interactive(children=(Dropdown(description='ds', options=('HR Clusters',), value='HR Clusters'), Dropdown(desc…